In [144]:
import pandas as pd
import re
import regex as re
import os
#sys.path.append('../')
#from utils import find_lines_with_player, name_wordgroups, remove_similar_rows_per_player
import numpy as np


In [145]:
#in utils. for joana
from difflib import SequenceMatcher

# Function which finds the lines where a players name is contained
def find_lines_with_player(dataframe, playerlist, n_lines = 0):
    
    # create empty df 
    df_complete = pd.DataFrame()

    # iterating over all players
    for player in playerlist:

        # get players first and last_name to include them in later sentence checks
        player_first_name, player_last_name = player.split()

        # just select player indiviual data
        df_player = dataframe[dataframe["player"] == player]
        df_player = df_player.reset_index(drop=True)

        # iterate over all data for the player
        for i in range(len(df_player)):

            # get the current record
            current_line = df_player['data'].iloc[i]
            # split up the records in lines
            lines = current_line.split('\\n')
            # create an empty string
            new_string = ''

            line_counter = 0
            # iterate over all lines in the record
            for line in lines:
                # if the playername can be found in the line add the line to the string
                if line.find(player) != -1:
                    new_string = new_string + line + " "
                    if line_counter <= 0:
                        line_counter = line_counter + n_lines
            
                elif line.find(player_first_name) != -1:
                    new_string = new_string + line + " "
                    if line_counter <= 0:
                        line_counter = line_counter + n_lines
        
                elif line.find(player_last_name) != -1:
                    new_string = new_string + line + " "
                    if line_counter <= 0:
                        line_counter = line_counter + n_lines
            
                elif line_counter >= 0:
                    new_string = new_string + line + " "
                    line_counter = line_counter-1
        
            # switch the previos record against the newly created string 
            df_player['data'].iloc[i] = new_string

        # add the new data to the Dataframe and return
        df_complete = pd.concat([df_complete, df_player], axis=0)
        
    return df_complete

def remove_similar_rows_per_player(df, playerlist, threshold=0.9):
    '''The procedure of deleting similiar articles needs to be done by each player because if an article writes about 
    # e.g. two players we want to keep it for both of the players'''

    # define empty df which will be returned in the end
    df_complete = pd.DataFrame()

    for player in playerlist:
        
        # create the df for the player
        df_player = df[df["player"] == player]
        df_player = df_player.reset_index(drop=True)
        column_as_df = pd.DataFrame(df_player['data'])


        
        # Compute similarity scores for each pair of rows
        similarity_scores = {}
        for i, row in column_as_df.iterrows():
            for j, other_row in column_as_df.iterrows():
                if i >= j:
                    continue
                score = SequenceMatcher(None, row, other_row).ratio()
                if score >= threshold:
                    similarity_scores[(i, j)] = score
        
        # Identify rows to remove
        rows_to_remove = []
        for (i, j), score in similarity_scores.items():
            if i not in rows_to_remove and j not in rows_to_remove:
                rows_to_remove.append(j if df_player.index[i] < df_player.index[j] else i)
        
        # Remove rows and concatenate df
        df_player = df_player.drop(rows_to_remove)
        df_complete = pd.concat([df_complete, df_player], axis=0)

        #return modified DataFrame
    return df_complete

def name_wordgroups(df):
    '''
    Function to match first and surname to just last name
    '''
    # create patterns which should be matched 
    # first lastname and firstname should both result in just lastname
    pattern_match2d = np.array([[r"\b(mitchel bakker|mitchel)\b", 'bakker'], 
                                [r"\b(xabi alonso|xabi)\b", 'alonso'], 
                                [r"\b(exequiel palacios|exequiel)\b", 'palacios'],
                                [r"\b(nadiem amiri|nadiem)\b", 'amiri'],
                                [r"\b(kerem demirbay|kerem)\b", 'demirbay'],
                                [r"\b(robert andrich|robert)\b", 'andrich'],
                                [r"\b(exequiel palacios|exequiel)\b", 'palacios'],
                                [r"\b(piero hincapie|piero)\b", 'hincapie'],
                                [r"\b(jeremie frimpong|jeremie)\b", 'frimpong'],
                                [r"\b(jonathan tah|jonathan)\b", 'tah'],
                                [r"\b(moussa diaby|moussa)\b", 'diaby'],
                                [r"\b(mykhaylo mudryk|mykhaylo)\b", 'mudryk'],
                                [r"\b(amine adli|amine)\b", 'adli'],
                                [r"\b(florian wirtz|florian)\b", 'wirtz'],
                                [r"\b(jose mourinho|jose)\b", 'mourinho'],     
                                #other wordgroups
                                [r"\b(europa league)\b", 'europaleague'],
                                [r"\b(champions league)\b", 'championsleague'],
                                [r"\b(bayer leverkusen|bayer|leverkusen|leverkusens)\b", 'bayerleverkusen']
                                ])

    # do the pattern matching for each player
    for pattern, player in pattern_match2d:
        df['data'] = df['data'].apply(lambda x: re.sub(pattern, str(player), str(x)))

    return df

In [146]:
def extract_sentence(df):
    player_info_sentences = []
    for idx, row in df.iterrows():
        player = row['player']
        entry = row['data']
        sentences = re.split('\. ', entry)
        found_sentence = False
        for sentence in sentences:
            if player in sentence:
                player_info_sentences.append(sentence)
                found_sentence = True
                break
        if not found_sentence:
            player_info_sentences.append('')
    
    df['sentence'] = player_info_sentences
    return df


In [147]:
# Load the CSV file into a DataFrame
df_de = pd.read_csv('../Preprocessing/data_clean/de_clean_1.csv')
df_en = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/en_clean_1.csv')
df_en2 = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/en_clean_2.csv')
df_es = pd.read_csv('../Preprocessing/data_clean/es_clean_1.csv')

In [148]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         363 non-null    object
 1   player       363 non-null    object
 2   language     363 non-null    object
 3   publishedAt  363 non-null    object
dtypes: object(4)
memory usage: 11.5+ KB


In [149]:
df_en2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 363 entries, 0 to 362
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         363 non-null    object
 1   player       363 non-null    object
 2   language     363 non-null    object
 3   publishedAt  363 non-null    object
dtypes: object(4)
memory usage: 11.5+ KB


In [150]:
# Keep paragraph where the player name is found
df_de = find_lines_with_player(df_de, df_de['player'].unique(),n_lines=1)
df_en = find_lines_with_player(df_en, df_en['player'].unique(),n_lines=1)
df_en2 = find_lines_with_player(df_en2, df_en2['player'].unique(),n_lines=1)
df_es = find_lines_with_player(df_es, df_es['player'].unique(),n_lines=1)

In [151]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         363 non-null    object
 1   player       363 non-null    object
 2   language     363 non-null    object
 3   publishedAt  363 non-null    object
dtypes: object(4)
memory usage: 14.2+ KB


In [152]:
# Extract sentence
df_de = extract_sentence(df_de)
df_en = extract_sentence(df_en)
df_en2 = extract_sentence(df_en2)
df_es = extract_sentence(df_es)

In [153]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363 entries, 0 to 20
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         363 non-null    object
 1   player       363 non-null    object
 2   language     363 non-null    object
 3   publishedAt  363 non-null    object
 4   sentence     363 non-null    object
dtypes: object(5)
memory usage: 17.0+ KB


In [154]:
# copy sentence column into data and remove sentence
df_de['data']= df_de['sentence']
df_de.drop('sentence', axis=1, inplace=True)
df_en['data']= df_en['sentence']
df_en.drop('sentence', axis=1, inplace=True)
df_en2['data']= df_en2['sentence']
df_en2.drop('sentence', axis=1, inplace=True)
df_es['data']= df_es['sentence']
df_es.drop('sentence', axis=1, inplace=True)

In [155]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         363 non-null    object
 1   player       363 non-null    object
 2   language     363 non-null    object
 3   publishedAt  363 non-null    object
dtypes: object(4)
memory usage: 14.2+ KB


In [156]:
# Remove the similiar rows (The Function is imported from utils on top)
df_de = remove_similar_rows_per_player(df_de, df_de['player'].unique())
df_en = remove_similar_rows_per_player(df_en, df_en['player'].unique())
df_en2 = remove_similar_rows_per_player(df_en2, df_en2['player'].unique())
df_es = remove_similar_rows_per_player(df_es, df_es['player'].unique())

In [157]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         143 non-null    object
 1   player       143 non-null    object
 2   language     143 non-null    object
 3   publishedAt  143 non-null    object
dtypes: object(4)
memory usage: 5.6+ KB


In [158]:
# Change names in the text to unify player name to last name
df_de = name_wordgroups(df_de)
df_en = name_wordgroups(df_en)
df_en2 = name_wordgroups(df_en2)
df_es = name_wordgroups(df_es)

In [159]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         143 non-null    object
 1   player       143 non-null    object
 2   language     143 non-null    object
 3   publishedAt  143 non-null    object
dtypes: object(4)
memory usage: 5.6+ KB


In [141]:
# delet empty rows
df_de = df_de.replace('', pd.NA)
df_de.dropna(inplace=True)

df_en = df_en.replace('', pd.NA)
df_en.dropna(inplace=True)

df_en2 = df_en2.replace('', pd.NA)
df_en2.dropna(inplace=True)

df_es = df_es.replace('', pd.NA)
df_es.dropna(inplace=True)

In [142]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         136 non-null    object
 1   player       136 non-null    object
 2   language     136 non-null    object
 3   publishedAt  136 non-null    object
dtypes: object(4)
memory usage: 5.3+ KB


In [143]:
df_en2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         135 non-null    object
 1   player       135 non-null    object
 2   language     135 non-null    object
 3   publishedAt  135 non-null    object
dtypes: object(4)
memory usage: 5.3+ KB


In [94]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'en_clean_1_sen.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_en.to_csv(file_path, index=False)

In [95]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'en_clean_2_sen.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_en2.to_csv(file_path, index=False)

In [30]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'de_clean_1-1.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_de.to_csv(file_path, index=False)

In [32]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'es_clean_1-1.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_es.to_csv(file_path, index=False)